<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# IBM Watson OpenScale and Batch Processing:<br>Apache Spark on Cloud Pak for Data with IBM Analytics Engine

This notebook must be run in the Python 3.x runtime environment. It requires Watson OpenScale service credentials.

The notebook configures Watson OpenScale to monitor any WML online deployment as a self managed subscription on Openscale. For self managed subscriptions, all the monitors are run as jobs on user provided spark engine. Use the notebook to enable quality, drift, fairness and explainability monitoring and run on-demand evaluations. Before you can run the notebook, you must have the following resources:

1. Configuration Archive generated using common configuration notebook [common configuration notebook](https://github.com/IBM/watson-openscale-samples/blob/main/Cloud%20Pak%20for%20Data/Batch%20Support/Configuration%20generation%20for%20OpenScale%20batch%20subscription.ipynb) and pre created WML deployment.
2. Payload, feedback, drifted transactions, explanations queue and result tables to be created or validated in an IBM DB2 storage. Please note, some of these maybe optional depending on your use-case.

## Contents

* [1. Setup](#setup)
* [2. Configure Watson OpenScale](#openscale)
* [3. Set up Subscription](#subscription)
* [4. Quality monitoring](#quality)
* [5. Drift monitoring](#drift)
* [6. Fairness monitoring](#fairness)
* [7. Explainability monitoring](#explainability)

## Setup <a name="setup"></a>

### Installing Required Libraries

First import some of the packages you need to use. After you finish installing the following software packages, restart the kernel.

### Import configuration archive/package

Configuration archive/package created using configuration notebook will be required to onboard model for monitoring in IBM Watson OpenScale. Provide path location of archive here.

Please note if you are executing this notebook in IBM Watson Studio, first upload the configuration archive/package to project and use provided code snippet to download it to local directory of this notebook.

In [ ]:
import warnings
import tarfile
warnings.filterwarnings("ignore")
%env PIP_DISABLE_PIP_VERSION_CHECK=1

# Note: Restart kernel after the dependencies are installed
!pip install --upgrade ibm-watson-openscale
!pip install "ibm_wos_utils==5.0.1.16"

# # Download "configuration_archive.tar.gz" from project to local directory and 
# extract the artifacts
# from ibm_watson_studio_lib import access_project_or_space
# wslib = access_project_or_space()
# wslib.download_file("configuration_archive.tar.gz")
archive_file_path = "configuration_archive.tar.gz"
with tarfile.open('configuration_archive.tar.gz', "r:gz") as tar:
    tar.extractall('.')
# On unpacking the configuration archive, the files [common_configuration.json, drift_archive.tar.gz, 
# explainability.tar.gz, fairness_statistics.json] will be extracted to the current directory. 
# Output may differ depending on which monitors were enabled while running the 
# common configuration notebook

In [ ]:
!pip show ibm-watson-openscale

## Configure credentials

Provide your IBM Watson OpenScale credentials in the following cell:

In [ ]:
WOS_CREDENTIALS = {
    "url": "<cluster-url>",
    "username": "<username>",
    "password": "<password>",
    "instance_id": "<openscale instance id>"
}

WML_CREDENTIALS = {
    "url": "<wml_url>",
    "username": "<username",
    "password": "<password>",
    "instance_id": "<instance id>"
}

## Specify model details

### Service provider and subscription metadata

In [ ]:
# Service Provider

SERVICE_PROVIDER_NAME = "<service-provider-name>"
SERVICE_PROVIDER_DESCRIPTION = "<service-provider-description>"

# Subscription

SUBSCRIPTION_NAME = "<subscription-name>"
SUBSCRIPTION_DESCRIPTION = "<subscription-description>"

### IBM Analytics Engine - Spark

Make sure that the Apache Spark manager on IBM Analytics Engine is running, and then provide the following details:

- IAE_SPARK_DISPLAY_NAME: _Display Name of the Spark instance in IBM Analytics Engine_
- IAE_SPARK_JOBS_ENDPOINT: _Spark Jobs Endpoint for IBM Analytics Engine_
- IBM_CPD_VOLUME: _IBM Cloud Pak for Data storage volume name_
- IBM_CPD_USERNAME: _IBM Cloud Pak for Data username_
- IBM_CPD_APIKEY: _IBM Cloud Pak for Data API key_
- IAE_SPARK_DESCRIPTION: _Custom description for the Spark instance_

In [ ]:
IAE_SPARK_DISPLAY_NAME = "<spark-engine-name>"
IAE_SPARK_JOBS_ENDPOINT = "<spark-job-endpoint-for-ibm-analytics-engine>"
IBM_CPD_VOLUME = "<ibm-cpd-volume>"
IBM_CPD_USERNAME = "<ibm-cloud-pak-for-data-username>"
IBM_CPD_APIKEY = "<ibm-cloud-pak-for-data-apikey>"
IAE_SPARK_NAME = "<iae-spark-name>"
IAE_SPARK_DESCRIPTION = "<iae-spark-description>"

#### Provide Spark Resource Settings

To configure how much of your Spark Cluster resources this job can consume, edit the following values:

- max_num_executors: _Maximum Number of executors to launch for this session_
- min_executors: _Minimum Number of executors to launch for this session_
- executor_cores: _Number of cores to use for each executor_   
- executor_memory: _Amount of memory (in GBs) to use per executor process_
- driver_cores: _Number of cores to use for the driver process_
- driver_memory: _Amount of memory (in GBs) to use for the driver process_

In [ ]:
spark_parameters = {
    "max_num_executors": 1,
    "min_num_executors": 1,
    "executor_cores": 1,
    "executor_memory": 1,
    "driver_cores": 1,
    "driver_memory": 1
}

### Storage Inputs

Please enter a name and description for your JDBC Storage

- JDBC_CONNECTION_NAME: _Custom display name for the JDBC Storage Connection_
- JDBC_CONNECTION_DESCRIPTION: _Custom description for the JDBC Storage Connection_

To connect to your JDBC storage, you must provide the following details:

 - JDBC_HOST: Hostname of the JDBC Connection
 - JDBC_PORT: Port of the JDBC Connection
 - JDBC_USE_SSL: Boolean Flag to indicate whether to use SSL while connecting.
 - JDBC_SSL_CERTIFICATE: SSL Certificate [Base64 encoded string] of the JDBC Connection. Ignored if JDBC_USE_SSL is False.
 - JDBC_DRIVER: Class name of the JDBC driver to use to connect.
 - JDBC_USERNAME: Username of the JDBC Connection
 - JDBC_PASSWORD: Password of the JDBC Connection
 - JDBC_DATABASE_NAME: Name of the Database to connect to.

In [ ]:
JDBC_CONNECTION_NAME = "<jdbc-connection-name>"
JDBC_CONNECTION_DESCRIPTION = "<jdbc-connection-description>"

JDBC_HOST = "<Hostname of the JDBC Connection>"
JDBC_PORT = "<Port of the JDBC Connection>"
JDBC_USE_SSL = "<Boolean Flag to indicate whether to use SSL while connecting.>"
JDBC_SSL_CERTIFICATE = "<SSL Certificate [Base64 encoded string] of the JDBC Connection. Ignored if JDBC_USE_SSL is False.>"
JDBC_DRIVER = "<Class name of the JDBC driver to use to connect.>"
JDBC_USERNAME = "<Username of the JDBC Connection>"
JDBC_PASSWORD = "<Password of the JDBC Connection>"
JDBC_DATABASE_NAME = "<Name of the Database to connect to.>"

JDBC_CONNECTION_NAME = "jdbc for batch parity"
JDBC_CONNECTION_DESCRIPTION = "jdbc connection from SDK"

In [ ]:
num_partitions_recommended = 12

if spark_parameters:
    executors = spark_parameters.get("max_num_executors", 2)
    cores = spark_parameters.get("executor_cores", 2)
    num_partitions_recommended = 2 * executors * cores
    
print("{} is the recommended value for number of partitions in your data. Please change this value as per your data.".format(num_partitions_recommended))

- **PARTITION_COLUMN**: The column to help Spark read and write data using multiple workers in your JDBC storage. This will help improve the performance of your Spark jobs. The default value is set to `wos_partition_column`.
- **NUM_PARTITIONS**: The maximum number of partitions that Spark can divide the data into. In JDBC, it also means the maximum number of connections that Spark can make to the JDBC store for reading/writing data. 

The recommended value is calculated in the above cell as a multiple of total workers allotted for this job. You can use the same value or change it in the next cell.

In [ ]:
PARTITION_COLUMN = "wos_partition_column"
NUM_PARTITIONS = num_partitions_recommended

### Feedback table metadata

The quality monitor stores metadata in the feedback table. To configure the quality monitor, you must provide the following details. To skip quality monitoring, run the following cell to initialize variables with the value of `None`.

- FEEDBACK_DATABASE_NAME: _Database name where feedback table is present_
- FEEDBACK_SCHEMA_NAME: _Schema name where feedback table is present_
- FEEDBACK_TABLE_NAME: _Name of the feedback table_

In [ ]:
#feedback

FEEDBACK_DATABASE_NAME = None
FEEDBACK_SCHEMA_NAME = None
FEEDBACK_TABLE_NAME = None

### Payload and drift table metadata

The drift monitor stores metadata in the payload and drift tables. To configure the drift monitor, you must provide the following details. To skip drift monitoring, run the following cell to initialize variables with the value of `None`.

- PAYLOAD_DATABASE_NAME: _Database name where payload logging table is present_
- PAYLOAD_SCHEMA_NAME: _Schema name where payload logging table is present_
- PAYLOAD_TABLE_NAME: _Name of the payload logging table_
- DRIFT_DATABASE_NAME: _Database name where drifted transactions table is present_
- DRIFT_SCHEMA_NAME: _Schema name where drifted transactions table is present_
- DRIFT_TABLE_NAME: _Name of the drifted transactions table_

In [ ]:
#payload logging

PAYLOAD_DATABASE_NAME = None
PAYLOAD_SCHEMA_NAME = None
PAYLOAD_TABLE_NAME = None

#drift

DRIFT_DATABASE_NAME = None
DRIFT_SCHEMA_NAME = None
DRIFT_TABLE_NAME = None

### Explainability table metadata

The explainability monitor requires the queue and result tables. The payload table can also be used as the queue table. To configure the explainability monitor, you must provide the following details. To skip explainability monitoring, run the following cell to initialize variables with the value of `None`.

- EXPLAINABILITY_DATABASE_NAME: _Database name where explanations queue, result tables are present_
- EXPLAINABILITY_QUEUE_TABLE_NAME: _Name of the explanations queue table_
- EXPLAINABILITY_RESULT_TABLE_NAME: _Name of the explanations result table_

In [ ]:
#explainability

EXPLAINABILITY_DATABASE_NAME = None
EXPLAINABILITY_SCHEMA_NAME = None
EXPLAINABILITY_QUEUE_TABLE_NAME = None
EXPLAINABILITY_RESULT_TABLE_NAME = None

# 2. Configure Watson OpenScale <a name="openscale"></a>

### Import the required libraries and set up the Watson OpenScale client

In [ ]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import *

authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS["url"],
        username=WOS_CREDENTIALS["username"],
        password=WOS_CREDENTIALS["password"],
        disable_ssl_verification=True
    )

wos_client = APIClient(authenticator=authenticator, service_url=WOS_CREDENTIALS["url"], service_instance_id=WOS_CREDENTIALS["instance_id"])
data_mart_id=WOS_CREDENTIALS["instance_id"]

### Display Watson OpenScale datamart details

In [ ]:
wos_client.data_marts.show()

### Create a service provider

In [ ]:
# Delete existing service provider with the same name as provided

service_providers = wos_client.service_providers.list().result.service_providers
for provider in service_providers:
    if provider.entity.name == SERVICE_PROVIDER_NAME:
        wos_client.service_providers.delete(service_provider_id=provider.metadata.id)
        break

In [ ]:
# Add Service Provider

added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        credentials=WML_CREDENTIALS,
        operational_space_id="production",
        background_mode=False
    ).result

service_provider_id = added_service_provider_result.metadata.id

wos_client.service_providers.show()

In [ ]:
service_provide_details = wos_client.service_providers.get(service_provider_id=service_provider_id).result
print(service_provide_details)

### Create integrated systems for Spark Engine and JDBC Storage

In [ ]:
# Delete existing spark and jdbc integrated systems if present

integrated_systems = IntegratedSystems(wos_client).list().result.integrated_systems

for system in integrated_systems:
    if system.entity.name in (IAE_SPARK_NAME, JDBC_CONNECTION_NAME):
        print("Deleting integrated system {}".format(system.entity.name))
        IntegratedSystems(wos_client).delete(integrated_system_id=system.metadata.id)

#### Spark Engine

In [ ]:
spark_integrated_system = IntegratedSystems(wos_client).add(
    name=IAE_SPARK_NAME,
    description=IAE_SPARK_DESCRIPTION,
    type="spark",
    credentials={
        "username": IBM_CPD_USERNAME,
        "apikey": IBM_CPD_APIKEY
    },
    connection={
        "display_name": IAE_SPARK_DISPLAY_NAME,
        "endpoint": IAE_SPARK_JOBS_ENDPOINT,
        "volume": IBM_CPD_VOLUME,
        "location_type": "cpd_iae"
    }
).result

spark_integrated_system_id = spark_integrated_system.metadata.id
print(spark_integrated_system)

#### JDBC Storage

In [ ]:
jdbc_url = "jdbc:db2://{}:{}/{}".format(JDBC_HOST, JDBC_PORT, JDBC_DATABASE_NAME)

jdbc_integrated_system = IntegratedSystems(wos_client).add(
    name=JDBC_CONNECTION_NAME,
    description=JDBC_CONNECTION_DESCRIPTION,
    type="jdbc",
    credentials={
        "username": JDBC_USERNAME,
        "password": JDBC_PASSWORD,
        "jdbc_url": jdbc_url
    },
    connection={
        "location_type": "jdbc",
        "db_driver": JDBC_DRIVER,
        "use_ssl": JDBC_USE_SSL,
        "certificate": JDBC_SSL_CERTIFICATE,
    }
).result

jdbc_integrated_system_id=jdbc_integrated_system.metadata.id
print(jdbc_integrated_system)

# 3. Set up a subscription<a name="subscription"></a>

In [ ]:
# Delete an existing subscription with the provided name

subscriptions = wos_client.subscriptions.list().result.subscriptions
for sub in subscriptions:
    if sub.entity.deployment.name == SUBSCRIPTION_NAME:
        wos_client.subscriptions.delete(subscription_id=sub.metadata.id)
        break

# Display all subscriptions
wos_client.subscriptions.show()

### Set subscription metadata

In the following cell, we expect you extracted Configuration Archive to current directory and it contains common_configuration.json. If not true, please update path to correct location. After you edit the path information, run the cell to set the asset details and properties, the deployment details, the analytics engine details, and to add the required tables as data sources.

In [ ]:
import uuid

# Provide the WML deployment io and space d
deployment_uid = "<wml_deployment_id>"
space_id = "<wml_space_id>"

common_configuration = None
with open("common_configuration.json", "r") as fp:
    configuration_json = json.load(fp)
    common_configuration = configuration_json.get("common_configuration")
    if common_configuration is None:
        raise Exception("Please provide the correct path to the common configuration JSON")
        
asset_deployment_details = wos_client.service_providers.list_assets(
    data_mart_id=data_mart_id, service_provider_id=service_provider_id, 
    deployment_id = deployment_uid, deployment_space_id = space_id).result['resources'][0]

model_asset_details_from_deployment = wos_client.service_providers.get_deployment_asset(
    data_mart_id=data_mart_id,service_provider_id=service_provider_id,deployment_id=deployment_uid,
    deployment_space_id=space_id)

scoring_endpoint = ScoringEndpointRequest(
    url=model_asset_details_from_deployment['entity']['scoring_endpoint']['url'] )

    
# Set asset details
asset = Asset(
    asset_id=model_asset_details_from_deployment["entity"]["asset"]["asset_id"],
    name=model_asset_details_from_deployment["entity"]["asset"]["name"],
    url=model_asset_details_from_deployment["entity"]["asset"]["url"],
    asset_type=AssetTypes.MODEL,
    input_data_type=InputDataType.STRUCTURED,
    problem_type=common_configuration.get("problem_type") if common_configuration.get("problem_type") else common_configuration.get("model_type")
)

# Set deployment details
asset_deployment = AssetDeploymentRequest(
    deployment_id=asset_deployment_details['metadata']['guid'],
    name=asset_deployment_details['entity']['name'],
    deployment_type= DeploymentTypes.ONLINE,
    url=asset_deployment_details['entity']['scoring_endpoint']['url'],
    scoring_endpoint=scoring_endpoint
)

probability_fields = common_configuration.get("probability_fields") if common_configuration.get("probability_fields") else [common_configuration["probability"]]

# Set asset properties 
asset_properties_request = AssetPropertiesRequest(
    label_column=common_configuration["label_column"],
    probability_fields=probability_fields, # comment out this line for regression models as probability_fields is not applicable
    prediction_field=common_configuration["prediction"],
    feature_fields=common_configuration["feature_columns"],
    categorical_fields=common_configuration["categorical_columns"]
)

In [ ]:
# Set analytics engine details
analytics_engine = AnalyticsEngine(
    type="spark",
    integrated_system_id=spark_integrated_system_id,
    parameters = spark_parameters
)

# Add selected tables as data sources
data_sources = []
if FEEDBACK_DATABASE_NAME is not None and FEEDBACK_TABLE_NAME is not None:
    feedback_data_source = DataSource(
        type="feedback", 
        database_name=FEEDBACK_DATABASE_NAME,
        schema_name=FEEDBACK_SCHEMA_NAME,
        table_name=FEEDBACK_TABLE_NAME, 
        auto_create=True, #set it to False if table already exists
        connection=DataSourceConnection(
            type="jdbc", 
            integrated_system_id=jdbc_integrated_system_id
        ),
        parameters={
            "partition_column": PARTITION_COLUMN,
            "num_partitions": NUM_PARTITIONS
        }
    )
    data_sources.append(feedback_data_source)
    
if PAYLOAD_DATABASE_NAME is not None and PAYLOAD_TABLE_NAME is not None \
    and DRIFT_DATABASE_NAME is not None and DRIFT_TABLE_NAME is not None:
    payload_logging_data_source = DataSource(
        type="payload", 
        database_name=PAYLOAD_DATABASE_NAME,
        schema_name=PAYLOAD_SCHEMA_NAME,
        table_name=PAYLOAD_TABLE_NAME, 
        auto_create=True, #set it to False if table already exists
        connection=DataSourceConnection(
            type="jdbc", 
            integrated_system_id=jdbc_integrated_system_id
        ),
        parameters={
            "partition_column": PARTITION_COLUMN,
            "num_partitions": NUM_PARTITIONS
        }
    )
    
    drifted_transactions_table_data_source = DataSource(
        type="drift", 
        database_name=DRIFT_DATABASE_NAME,
        schema_name=DRIFT_SCHEMA_NAME,
        table_name=DRIFT_TABLE_NAME,
        auto_create=True, #set it to False if table already exists
        connection=DataSourceConnection(
            type="jdbc", 
            integrated_system_id=jdbc_integrated_system_id
        ),
        parameters={
        
        }
    )
    
    data_sources.append(payload_logging_data_source)
    data_sources.append(drifted_transactions_table_data_source)

if EXPLAINABILITY_DATABASE_NAME is not None and EXPLAINABILITY_QUEUE_TABLE_NAME is not None:
    explainability_queue_data_source = DataSource(
        type="explain_queue", 
        database_name=EXPLAINABILITY_DATABASE_NAME,
        schema_name=EXPLAINABILITY_SCHEMA_NAME,
        table_name=EXPLAINABILITY_QUEUE_TABLE_NAME, 
        auto_create=True, #set it to False if table already exists
        connection=DataSourceConnection(
            type="jdbc", 
            integrated_system_id=jdbc_integrated_system_id
        ),
        parameters={
            "partition_column": PARTITION_COLUMN,
            "num_partitions": NUM_PARTITIONS
        }
    )
    
    data_sources.append(explainability_queue_data_source)

if EXPLAINABILITY_DATABASE_NAME is not None and EXPLAINABILITY_RESULT_TABLE_NAME is not None:
    explainability_result_data_source = DataSource(
        type="explain_result", 
        database_name=EXPLAINABILITY_DATABASE_NAME,
        schema_name=EXPLAINABILITY_SCHEMA_NAME,
        table_name=EXPLAINABILITY_RESULT_TABLE_NAME, 
        auto_create=True, #set it to False if table already exists
        connection=DataSourceConnection(
            type="jdbc", 
            integrated_system_id=jdbc_integrated_system_id
        ),
        parameters={
        
        }
    )
    
    data_sources.append(explainability_result_data_source)

In [ ]:
# Adding the subscription

subscription_details = Subscriptions(wos_client).add(
    data_mart_id=data_mart_id,
    service_provider_id=service_provider_id,
    asset=asset,
    deployment=asset_deployment,
    asset_properties=asset_properties_request,
    analytics_engine=analytics_engine,
    scoring_endpoint=scoring_endpoint).result

subscription_id = subscription_details.metadata.id
print(subscription_details)

In [ ]:
import time
# Checking subscription status

state = wos_client.subscriptions.get(subscription_id).result.entity.status.state

while state not in ["active", "error"]:
    state = wos_client.subscriptions.get(subscription_id).result.entity.status.state
    print(state)
    time.sleep(5)

In [ ]:
# Add training, output, and input data schemas to the subscription
schemas_patch_document = [
    JsonPatchOperation(op=OperationTypes.REPLACE, path='/asset_properties/training_data_schema', value=common_configuration["training_data_schema"]),
    JsonPatchOperation(op=OperationTypes.REPLACE, path='/asset_properties/input_data_schema', value=common_configuration["input_data_schema"]),
    JsonPatchOperation(op=OperationTypes.REPLACE, path='/asset_properties/output_data_schema', value=common_configuration["output_data_schema"])
]

wos_client.subscriptions.update(subscription_id=subscription_id, patch_document=schemas_patch_document)

# Add data_sources to the subscription
data_sources_patch_document=[
    JsonPatchOperation(op=OperationTypes.ADD, path='/data_sources', value=[data_source.to_dict() for data_source in data_sources])
]

wos_client.subscriptions.update(subscription_id=subscription_id, patch_document=data_sources_patch_document)

In [ ]:
import time

# Checking subscription status after the patch operation

state = wos_client.subscriptions.get(subscription_id).result.entity.status.state

while state not in ["active", "error"]:
    state = wos_client.subscriptions.get(subscription_id).result.entity.status.state
    print(state)
    time.sleep(5)

# 4. Quality monitoring <a name="quality"></a>

### Enable the quality monitor

In the following code cell, default values are set for the quality monitor. You can change the default values by updating the optional `min_feedback_data_size` attribute in the `parameters` dict and set the quality threshold in the `thresholds` list.

In [ ]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    "min_feedback_data_size": 1000
}

thresholds = [{
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
}]

quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

quality_monitor_instance_id = quality_monitor_details.metadata.id
print(quality_monitor_details)

### Check monitor instance status

In [ ]:
quality_status = None

while quality_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=quality_monitor_instance_id).result
    quality_status = monitor_instance_details.entity.status.state
    if quality_status not in ("active", "error"):
        print(datetime.now().strftime("%H:%M:%S"), quality_status)
        time.sleep(30)
        
print(datetime.now().strftime("%H:%M:%S"), quality_status)

In [ ]:
monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=quality_monitor_instance_id).result
print(monitor_instance_details)

### Run an on-demand evaluation

Please make sure you have data in feedback table in db2. Quality evaluation will use this data for its metrics computation.

In [ ]:
# Trigger on-demand run
monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

In [ ]:
# Check run status

quality_run_status = None
while quality_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=quality_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    quality_run_status = monitoring_run_details.entity.status.state
    if quality_run_status not in ("finished", "error"):
        print(datetime.now().strftime("%H:%M:%S"), quality_run_status)
        time.sleep(30)
        
print(datetime.now().strftime("%H:%M:%S"), quality_run_status)

### Display quality metrics

In [ ]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

# 5. Drift monitoring <a name="drift"></a>

### Enable the drift monitor

In the following code cell, type a path to the drift configuration tar ball.

In [ ]:
wos_client.monitor_instances.upload_drift_model(
    model_path="drift_archive.tar.gz",
    data_mart_id=data_mart_id,
    subscription_id=subscription_id
).result

In the following code cell, default values are set for the drift monitor. You can change the default values by updating the values in the `parameters` section. The `min_samples` parameter controls the number of minimum records required in an evaluation time window for drift analysis to be completed. The `drift_threshold` parameter sets the threshold in decimal format for the drift percentage to trigger an alert. The `train_drift_model` parameter controls whether to learn drift artefacts (drift model training and data constraints learning) by IBM Watson OpenScale via Spark job accessing training data table.

In [ ]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    "min_samples": 1000,
    "drift_threshold": 0.05,
    "train_drift_model": False
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
print(drift_monitor_details)

### Check monitor instance status

In [ ]:
drift_status = None

while drift_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_monitor_instance_id).result
    drift_status = monitor_instance_details.entity.status.state
    if drift_status not in ("active", "error"):
        print(datetime.now().strftime("%H:%M:%S"), drift_status)
        time.sleep(30)

print(datetime.now().strftime("%H:%M:%S"), drift_status)

### Run an on-demand evaluation
Please make sure you have data in payload table in db2. Drift evaluation will use this data for its metrics computation.

In [ ]:
# Check Drift monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_monitor_instance_id).result
print(monitor_instance_details)

In [ ]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

In [ ]:
# Check run status

drift_run_status = None
while drift_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=drift_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    drift_run_status = monitoring_run_details.entity.status.state
    if drift_run_status not in ("finished", "error"):
        print(datetime.now().strftime("%H:%M:%S"), drift_run_status)
        time.sleep(30)
        
print(datetime.now().strftime("%H:%M:%S"), drift_run_status)

### Display drift metrics

In [ ]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

# 6. Fairness monitoring <a name="fairness"></a>

### Enable the fairness monitor

The following code cell, will enable the fairness monitor.

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

fairness_statistics_json = None
with open("fairness_statistics.json", "r") as fp:
    fairness_statistics_json = json.load(fp)
    
parameters = fairness_statistics_json["parameters"]
thresholds = fairness_statistics_json["thresholds"]

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

fairness_monitor_instance_id = fairness_monitor_details.metadata.id
print(fairness_monitor_details)

### Check monitor instance status

In [ ]:
fairness_state = fairness_monitor_details.entity.status.state

while fairness_state not in ("active", "error"):
    print(datetime.utcnow().strftime('%H:%M:%S'), fairness_state)
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=fairness_monitor_instance_id).result
    fairness_state = monitor_instance_details.entity.status.state
    time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), fairness_state)

### Run an on-demand evaluation
Please make sure you have data in payload table in db2. Fairness evaluation will use this data for its metrics computation.

In [ ]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

In [ ]:
# Check run status

fairness_run_status = monitoring_run_details.entity.status.state
while fairness_run_status not in ("finished", "error"):
    print(datetime.utcnow().strftime("%H:%M:%S"), fairness_run_status)
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=fairness_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    fairness_run_status = monitoring_run_details.entity.status.state
    time.sleep(30)
        
print(datetime.utcnow().strftime("%H:%M:%S"), fairness_run_status)

### Display fairness metrics

In [ ]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

# 7. Explainability monitoring <a name="explainability"></a>

### Enable the explainability monitor

#### Upload explainability configuration archive
In the following code cell, type the path to the explainability configuration archive tar ball.

In [ ]:
with open("explainability.tar.gz", mode="rb") as explainability_tar:
    wos_client.monitor_instances.upload_explainability_archive(subscription_id=subscription_id, archive=explainability_tar)

print("Uploaded explainability archive successfully.")

In [ ]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {}

explainability_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_instance_id = explainability_monitor_details.metadata.id
print(explainability_monitor_details)

### Check monitor instance status

In [ ]:
explainability_status = None

while explainability_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=explainability_monitor_instance_id).result
    explainability_status = monitor_instance_details.entity.status.state
    if explainability_status not in ("active", "error"):
        print(datetime.utcnow().strftime('%H:%M:%S'), explainability_status)
        time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), explainability_status)

### Run an on-demand evaluation
Please make sure you have data in Explain queue table in db2. Explainability will use this data for computing explanations.

In [ ]:
# Check Explainbility monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=explainability_monitor_instance_id).result
print(monitor_instance_details)

In [ ]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=explainability_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

In [ ]:
# Check run status

explainability_run_status = None
while explainability_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=explainability_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    explainability_run_status = monitoring_run_details.entity.status.state
    if explainability_run_status not in ("finished", "error"):
        print(datetime.utcnow().strftime("%H:%M:%S"), explainability_run_status)
        time.sleep(60)
        
print(datetime.utcnow().strftime("%H:%M:%S"), explainability_run_status)
print(monitoring_run_details)

### Display sample explanations

In [ ]:
explanations = wos_client.monitor_instances.get_all_explaination_tasks(subscription_id=subscription_id).result.to_dict()
print(explanations)

### Generate on demand explanation

Choose any record from payload logging table or Explain Queue table and generate Lime, Contrastive explanations for the record. Please note that contrastive explanations are only supported for structured classification models. So specify the explanation_types accordingly. 

In [ ]:
import time
# Please enter scoring id(from payload table or Explain Queue table), for which you want to generate an explanation
# scoring_ids = ["<scoring_id_from_pl_or_explain_queue_table>"] 

# As an example, we will generate a contrastive explanation for the first scoring id from the list of explanations generated above
# as part of the Explainability monitor run
scoring_ids = [explanations["explanation_values"][0][1]]
print("Running explanations on scoring IDs: {}".format(scoring_ids))

explanation_types = ["lime", "contrastive"]
explanation_task_ids = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, subscription_id=subscription_id, explanation_types=explanation_types).result.metadata.explanation_task_ids
explanation_task_id = explanation_task_ids[0]

print("Getting explanation for explanation task id: {}".format(explanation_task_id))
explanation = wos_client.monitor_instances.get_explanation_tasks(explanation_task_id, subscription_id=subscription_id).result
while explanation.entity.status.state not in ("finished", "error"):
        explanation = wos_client.monitor_instances.get_explanation_tasks(explanation_task_id, subscription_id=subscription_id).result
        explanation_run_status = explanation.entity.status.state
        if explanation_run_status not in ("finished", "error"):
            print(datetime.utcnow().strftime("%H:%M:%S"), explanation_run_status)
            time.sleep(60)
            
            
print(explanation)

### Cleanup the untarred files

In [ ]:
import os
files = ["common_configuration.json", "explainability.tar.gz", "drift_archive.tar.gz", "fairness_statistics.json"]
for file in files:
    if os.path.isfile(file):
        os.remove(file)

## Congratulations!

You have finished the Batch demo for IBM Watson OpenScale by using Apache Spark on Cloud Pak for Data IBM Analytics Engine. You can now view the [Watson OpenScale Dashboard](https://url-to-your-cp4d-cluster/aiopenscale). Click the tile for the model you configured to see the quality, drift and fairness monitors. Click the timeseries graph to get detailed information on transactions during a specific time window.